In [1]:
import requests
from bs4 import BeautifulSoup
import re
import sklearn
import time
import numpy as np
import json
import pandas as pd
pd.set_option("max_colwidth", 400)
# import sys   
# sys.setrecursionlimit(25000)

from pandarallel import pandarallel

In [2]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
from selenium.webdriver.chrome.options import Options
# from icecream import ic
from selenium.common.exceptions import TimeoutException, NoSuchElementException, ElementNotInteractableException
import os

chrome_options = Options()
chrome_options.headless = True
chrome_drive = './chromedriver'
os.environ['webdrive.chrome.drive'] = chrome_drive

In [3]:
def get_search_urls(date):
    return \
    {'us': 'https://www.nytimes.com/search?dropmab=true&endDate=%s&query=&sections=U.S.%%7Cnyt%%3A%%2F%%2Fsection%%2Fa34d3d6c-c77f-5931-b951-241b4e28681c&sort=best&startDate=%s&types=article'%(date,date),
     'world':'https://www.nytimes.com/search?dropmab=true&endDate=%s&query=&sections=World%%7Cnyt%%3A%%2F%%2Fsection%%2F70e865b6-cc70-5181-84c9-8368b3a5c34b&sort=best&startDate=%s&types=article'%(date,date)
    }

In [4]:
import datetime
start_dt = datetime.date(2012, 1, 1)
end_dt = datetime.date(2021, 3, 18)
date_generated = [start_dt + datetime.timedelta(days=x) for x in range(0, (end_dt-start_dt).days)]

date_generated = [date.strftime("%Y%m%d") for date in date_generated]
df = pd.DataFrame(date_generated, columns = ['date'])
df['urls']= df['date'].apply(get_search_urls)
df = pd.concat([df.drop(columns = 'urls'),pd.json_normalize(df.urls)], axis=1)
df = pd.melt(df, id_vars=['date'], value_vars=['us','world'])
df.columns = ['date','section','urls']
df = df.sort_values('date').reset_index(drop=True)
df

,date,section,urls
0,20120101,us,https://www.nytimes.com/search?dropmab=true&endDate=20120101&query=&sections=U.S.%7Cnyt%3A%2F%2Fsection%2Fa34d3d6c-c77f-5931-b951-241b4e28681c&sort=best&startDate=20120101&types=article
1,20120101,world,https://www.nytimes.com/search?dropmab=true&endDate=20120101&query=&sections=World%7Cnyt%3A%2F%2Fsection%2F70e865b6-cc70-5181-84c9-8368b3a5c34b&sort=best&startDate=20120101&types=article
2,20120102,us,https://www.nytimes.com/search?dropmab=true&endDate=20120102&query=&sections=U.S.%7Cnyt%3A%2F%2Fsection%2Fa34d3d6c-c77f-5931-b951-241b4e28681c&sort=best&startDate=20120102&types=article
3,20120102,world,https://www.nytimes.com/search?dropmab=true&endDate=20120102&query=&sections=World%7Cnyt%3A%2F%2Fsection%2F70e865b6-cc70-5181-84c9-8368b3a5c34b&sort=best&startDate=20120102&types=article
4,20120103,us,https://www.nytimes.com/search?dropmab=true&endDate=20120103&query=&sections=U.S.%7Cnyt%3A%2F%2Fsection%2Fa34d3d6c-c77f-5931-b951-241b4e28681c&sort=best&startDate=20120103&types=article
...,...,...,...
6723,20210315,us,https://www.nytimes.com/search?dropmab=true&endDate=20210315&query=&sections=U.S.%7Cnyt%3A%2F%2Fsection%2Fa34d3d6c-c77f-5931-b951-241b4e28681c&sort=best&startDate=20210315&types=article
6724,20210316,world,https://www.nytimes.com/search?dropmab=true&endDate=20210316&query=&sections=World%7Cnyt%3A%2F%2Fsection%2F70e865b6-cc70-5181-84c9-8368b3a5c34b&sort=best&startDate=20210316&types=article
6725,20210316,us,https://www.nytimes.com/search?dropmab=true&endDate=20210316&query=&sections=U.S.%7Cnyt%3A%2F%2Fsection%2Fa34d3d6c-c77f-5931-b951-241b4e28681c&sort=best&startDate=20210316&types=article
6726,20210317,us,https://www.nytimes.com/search?dropmab=true&endDate=20210317&query=&sections=U.S.%7Cnyt%3A%2F%2Fsection%2Fa34d3d6c-c77f-5931-b951-241b4e28681c&sort=best&startDate=20210317&types=article


In [5]:
def get_rows(url):
    driver = webdriver.Chrome(options=chrome_options, executable_path=chrome_drive)
    driver.get(url)

    while True:
        try:
            next_find = driver.find_element_by_xpath("//*[contains(text(), 'Show More')]")
            next_find.click()
            time.sleep(1)
        except:
            break        
    html = driver.page_source
    driver.quit()

    soup = BeautifulSoup(html, 'html.parser')
    rows = soup.find_all('li', class_= 'css-1l4w6pd')
    return [str(row) for row in rows]

In [6]:
pandarallel.initialize(nb_workers=20, progress_bar=True)
df[['rows']]= df['urls'].parallel_apply(get_rows)

INFO: Pandarallel will run on 20 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [15]:
df.loc[df.rows.str.len()==0, 'rows'] = df.loc[df.rows.str.len()==0, 'urls'].parallel_apply(get_rows)

In [18]:
df = df[df.rows.str.len()>0].reset_index(drop=True)
df.shape

(6724, 4)

In [ ]:
# df['rows'] = None

# pandarallel.initialize(nb_workers=24)
# batch_size = 96
# i=0
# while i<= df.shape[0]:
#     print(i, end=',')
#     df.loc[i: i+ batch_size-1, 'rows']= df.loc[i: i+ batch_size-1, 'urls'].parallel_apply(get_rows).values
#     i += batch_size

In [19]:
df= df.explode('rows').reset_index(drop=True)
df

,date,section,urls,rows
0,20120101,us,https://www.nytimes.com/search?dropmab=true&endDate=20120101&query=&sections=U.S.%7Cnyt%3A%2F%2Fsection%2Fa34d3d6c-c77f-5931-b951-241b4e28681c&sort=best&startDate=20120101&types=article,"<li class=""css-1l4w6pd"" data-testid=""search-bodega-result""><div class=""css-1kl114x""><span aria-label=""January 1, 2012"" class=""css-17ubb9w"" data-testid=""todays-date"">Jan. 1, 2012</span><div class=""css-1i8vfl5""><div class=""css-e1lvw9""><p class=""css-myxawk"">Politics</p><a href=""https://thecaucus.blogs.nytimes.com/2012/01/01/virginia-attorney-general-backs-off-ballot-proposal/?searchResultPosition..."
1,20120101,us,https://www.nytimes.com/search?dropmab=true&endDate=20120101&query=&sections=U.S.%7Cnyt%3A%2F%2Fsection%2Fa34d3d6c-c77f-5931-b951-241b4e28681c&sort=best&startDate=20120101&types=article,"<li class=""css-1l4w6pd"" data-testid=""search-bodega-result""><div class=""css-1kl114x""><span aria-label=""January 1, 2012"" class=""css-17ubb9w"" data-testid=""todays-date"">Jan. 1, 2012</span><div class=""css-1i8vfl5""><div class=""css-e1lvw9""><p class=""css-myxawk"">Politics</p><a href=""https://thecaucus.blogs.nytimes.com/2012/01/01/gingrich-says-hes-been-romney-boated/?searchResultPosition=2""><h4 class=""..."
2,20120101,us,https://www.nytimes.com/search?dropmab=true&endDate=20120101&query=&sections=U.S.%7Cnyt%3A%2F%2Fsection%2Fa34d3d6c-c77f-5931-b951-241b4e28681c&sort=best&startDate=20120101&types=article,"<li class=""css-1l4w6pd"" data-testid=""search-bodega-result""><div class=""css-1kl114x""><span aria-label=""January 1, 2012"" class=""css-17ubb9w"" data-testid=""todays-date"">Jan. 1, 2012</span><div class=""css-1i8vfl5""><div class=""css-e1lvw9""><p class=""css-myxawk"">Politics</p><a href=""https://thecaucus.blogs.nytimes.com/2012/01/01/romney-talks-about-santorum-a-bit-mischievously/?searchResultPosition=3"">..."
3,20120101,us,https://www.nytimes.com/search?dropmab=true&endDate=20120101&query=&sections=U.S.%7Cnyt%3A%2F%2Fsection%2Fa34d3d6c-c77f-5931-b951-241b4e28681c&sort=best&startDate=20120101&types=article,"<li class=""css-1l4w6pd"" data-testid=""search-bodega-result""><div class=""css-1kl114x""><span aria-label=""January 1, 2012"" class=""css-17ubb9w"" data-testid=""todays-date"">Jan. 1, 2012</span><div class=""css-1i8vfl5""><div class=""css-e1lvw9""><p class=""css-myxawk"">Politics</p><a href=""https://thecaucus.blogs.nytimes.com/2012/01/01/santorum-on-the-rise-pushes-his-supporters-for-help/?searchResultPosition..."
4,20120101,us,https://www.nytimes.com/search?dropmab=true&endDate=20120101&query=&sections=U.S.%7Cnyt%3A%2F%2Fsection%2Fa34d3d6c-c77f-5931-b951-241b4e28681c&sort=best&startDate=20120101&types=article,"<li class=""css-1l4w6pd"" data-testid=""search-bodega-result""><div class=""css-1bdu3ax""><span aria-label=""January 1, 2012"" class=""css-17ubb9w"" data-testid=""todays-date"">Jan. 1, 2012</span><div class=""css-1i8vfl5""><div class=""css-e1lvw9""><p class=""css-myxawk"">Politics</p><a href=""https://thecaucus.blogs.nytimes.com/2012/01/01/gingrich-plans-to-target-romney-in-new-hampshire/?searchResultPosition=5""..."
...,...,...,...,...
132892,20210317,world,https://www.nytimes.com/search?dropmab=true&endDate=20210317&query=&sections=World%7Cnyt%3A%2F%2Fsection%2F70e865b6-cc70-5181-84c9-8368b3a5c34b&sort=best&startDate=20210317&types=article,"<li class=""css-1l4w6pd"" data-testid=""search-bodega-result""><div class=""css-1kl114x""><span aria-label=""March 17"" class=""css-17ubb9w"" data-testid=""todays-date"">March 17</span><div class=""css-1i8vfl5""><div class=""css-e1lvw9""><p class=""css-myxawk"">Asia Pacific</p><a href=""/2021/03/17/world/asia/japan-gay-marriage.html?searchResultPosition=16""><h4 class=""css-2fgx4k"">Landmark Ruling Cracks Door Open..."
132893,20210317,world,https://www.nytimes.com/search?dropmab=true&endDate=20210317&query=&sections=World%7Cnyt%3A%2F%2Fsection%2F70e865b6-cc70-5181-84c9-8368b3a5c34b&sort=best&startDate=20210317&types=article,"<li 

In [20]:
df.to_csv('nytimes_raw_2012_2021.csv',index=False)

In [21]:
def extract_rows(row):
    soup = BeautifulSoup(row, 'html.parser')
    try:
        a = soup.find_all('a', href=True)[0]
        p = a.find_all('p')
        href = a['href']
        href = 'https://www.nytimes.com' + href if href.startswith('/') else href

        date = soup.span['aria-label']
        sec = soup.p.text
        title = a.h4.text
        if len(p)==2:
            abstract = p[0].text
            author = p[1].text.replace('By ', '')
        elif len(p)==1:
            abstract = 'None'
            author = p[0].text.replace('By ', '') 
        else:
            abstract = 'None'
            author = 'None'
        return pd.Series([date, sec, title, abstract, author, href])
    except:
        print('error')
        return pd.Series(['error']*6)

In [22]:
pandarallel.initialize()
df[['story_date', 'sec', 'title', 'abstract', 'author', 'href']]= df['rows'].parallel_apply(extract_rows).values

INFO: Pandarallel will run on 24 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
error


In [23]:
df.sec.unique()

array(['Politics', 'U.S.', 'Middle East', 'Asia Pacific', 'Europe',
       'Africa', 'World', 'Americas', 'Canada', 'Australia',
       'What in the World', 'Elections', 'error'], dtype=object)

In [26]:
df = df[~df.sec.isin(['What in the World','error'])].reset_index(drop=True)
df.head()

,date,section,urls,rows,story_date,sec,title,abstract,author,href
0,20120101,us,https://www.nytimes.com/search?dropmab=true&endDate=20120101&query=&sections=U.S.%7Cnyt%3A%2F%2Fsection%2Fa34d3d6c-c77f-5931-b951-241b4e28681c&sort=best&startDate=20120101&types=article,"<li class=""css-1l4w6pd"" data-testid=""search-bodega-result""><div class=""css-1kl114x""><span aria-label=""January 1, 2012"" class=""css-17ubb9w"" data-testid=""todays-date"">Jan. 1, 2012</span><div class=""css-1i8vfl5""><div class=""css-e1lvw9""><p class=""css-myxawk"">Politics</p><a href=""https://thecaucus.blogs.nytimes.com/2012/01/01/virginia-attorney-general-backs-off-ballot-proposal/?searchResultPosition...","January 1, 2012",Politics,Virginia Attorney General Backs Off Ballot Proposal,Virginia’s attorney general on Sunday backed off of a proposal to loosen the state’s ballot access rules to allow more Republican presidential candidates to qualify.,Michael D. Shear,https://thecaucus.blogs.nytimes.com/2012/01/01/virginia-attorney-general-backs-off-ballot-proposal/?searchResultPosition=1
1,20120101,us,https://www.nytimes.com/search?dropmab=true&endDate=20120101&query=&sections=U.S.%7Cnyt%3A%2F%2Fsection%2Fa34d3d6c-c77f-5931-b951-241b4e28681c&sort=best&startDate=20120101&types=article,"<li class=""css-1l4w6pd"" data-testid=""search-bodega-result""><div class=""css-1kl114x""><span aria-label=""January 1, 2012"" class=""css-17ubb9w"" data-testid=""todays-date"">Jan. 1, 2012</span><div class=""css-1i8vfl5""><div class=""css-e1lvw9""><p class=""css-myxawk"">Politics</p><a href=""https://thecaucus.blogs.nytimes.com/2012/01/01/gingrich-says-hes-been-romney-boated/?searchResultPosition=2""><h4 class=""...","January 1, 2012",Politics,Gingrich Says He’s Been ‘Romney-Boated’,Newt Gingrich promised an aggressive and pointed response in New Hampshire and South Carolina to attack advertisements in Iowa by groups supporting Mitt Romney.,Trip Gabriel,https://thecaucus.blogs.nytimes.com/2012/01/01/gingrich-says-hes-been-romney-boated/?searchResultPosition=2
2,20120101,us,https://www.nytimes.com/search?dropmab=true&endDate=20120101&query=&sections=U.S.%7Cnyt%3A%2F%2Fsection%2Fa34d3d6c-c77f-5931-b951-241b4e28681c&sort=best&startDate=20120101&types=article,"<li class=""css-1l4w6pd"" data-testid=""search-bodega-result""><div class=""css-1kl114x""><span aria-label=""January 1, 2012"" class=""css-17ubb9w"" data-testid=""todays-date"">Jan. 1, 2012</span><div class=""css-1i8vfl5""><div class=""css-e1lvw9""><p class=""css-myxawk"">Politics</p><a href=""https://thecaucus.blogs.nytimes.com/2012/01/01/romney-talks-about-santorum-a-bit-mischievously/?searchResultPosition=3"">...","January 1, 2012",Politics,"Romney Talks About Santorum, a Bit Mischievously",Mitt Romney notes that Rick Santorum endorsed him in 2008.,Ashley Parker,https://thecaucus.blogs.nytimes.com/2012/01/01/romney-talks-about-santorum-a-bit-mischievously/?searchResultPosition=3
3,20120101,us,https://www.nytimes.com/search?dropmab=true&endDate=20120101&query=&sections=U.S.%7Cnyt%3A%2F%2Fsection%2Fa34d3d6c-c77f-5931-b951-241b4e28681c&sort=best&startDate=20120101&types=article,"<li class=""css-1l4w6pd"" data-testid=""search-bodega-result""><div class=""css-1kl114x""><span aria-label=""January 1, 2012"" class=""css-17ubb9w"" data-testid=""todays-date"">Jan. 1, 2012</span><div class=""css-1i8vfl5""><div class=""css-e1lvw9""><p class=""css-myxawk"">Politics</p><a href=""https://thecaucus.blogs.nytimes.com/2012/01/01/santorum-on-the-rise-pushes-his-supporters-for-help/?searchResultPosition...","January 1, 2012",Politics,Santorum Pushes Supporters to Keep His Surge Alive,A question of whether his sudden gains in the polls will hold.,Jeremy W. Peters,https://thecaucus.blogs.nytimes.com/2012/01/01/santorum-on-the-rise-pushes-his-supporters-for-help/?searchResultPosition=4
4,20120101,us,https://www.nytimes.com/search?dropmab=true&endDate=20120101&query=&sections=U.S.%7Cnyt%3A%2F%2Fsection%2Fa34d3d6c-c77f-5931-b951-241b4e28681c&sort=

In [27]:
df = df[~df.duplicated(subset='href')].reset_index(drop=True)
df.shape

(132503, 10)

In [28]:
df = df.iloc[:,4:]
df.head()

,story_date,sec,title,abstract,author,href
0,"January 1, 2012",Politics,Virginia Attorney General Backs Off Ballot Proposal,Virginia’s attorney general on Sunday backed off of a proposal to loosen the state’s ballot access rules to allow more Republican presidential candidates to qualify.,Michael D. Shear,https://thecaucus.blogs.nytimes.com/2012/01/01/virginia-attorney-general-backs-off-ballot-proposal/?searchResultPosition=1
1,"January 1, 2012",Politics,Gingrich Says He’s Been ‘Romney-Boated’,Newt Gingrich promised an aggressive and pointed response in New Hampshire and South Carolina to attack advertisements in Iowa by groups supporting Mitt Romney.,Trip Gabriel,https://thecaucus.blogs.nytimes.com/2012/01/01/gingrich-says-hes-been-romney-boated/?searchResultPosition=2
2,"January 1, 2012",Politics,"Romney Talks About Santorum, a Bit Mischievously",Mitt Romney notes that Rick Santorum endorsed him in 2008.,Ashley Parker,https://thecaucus.blogs.nytimes.com/2012/01/01/romney-talks-about-santorum-a-bit-mischievously/?searchResultPosition=3
3,"January 1, 2012",Politics,Santorum Pushes Supporters to Keep His Surge Alive,A question of whether his sudden gains in the polls will hold.,Jeremy W. Peters,https://thecaucus.blogs.nytimes.com/2012/01/01/santorum-on-the-rise-pushes-his-supporters-for-help/?searchResultPosition=4
4,"January 1, 2012",Politics,Gingrich to Take Aim at Romney in New Hampshire,Newt Gingrich said on Sunday he intended to campaign in New Hampshire and will criticize Mitt Romney over an abortion provision in the Massachusetts health care program.,Katharine Q. Seelye,https://thecaucus.blogs.nytimes.com/2012/01/01/gingrich-plans-to-target-romney-in-new-hampshire/?searchResultPosition=5


In [29]:
df.to_csv('nytimes_links_2012_2021.csv',index=False)

In [2]:
df=pd.read_csv('nytimes_links_2012_2021.csv',header=[0])
df

,story_date,sec,title,abstract,author,href
0,"January 1, 2012",Politics,Virginia Attorney General Backs Off Ballot Proposal,Virginia’s attorney general on Sunday backed off of a proposal to loosen the state’s ballot access rules to allow more Republican presidential candidates to qualify.,Michael D. Shear,https://thecaucus.blogs.nytimes.com/2012/01/01/virginia-attorney-general-backs-off-ballot-proposal/?searchResultPosition=1
1,"January 1, 2012",Politics,Gingrich Says He’s Been ‘Romney-Boated’,Newt Gingrich promised an aggressive and pointed response in New Hampshire and South Carolina to attack advertisements in Iowa by groups supporting Mitt Romney.,Trip Gabriel,https://thecaucus.blogs.nytimes.com/2012/01/01/gingrich-says-hes-been-romney-boated/?searchResultPosition=2
2,"January 1, 2012",Politics,"Romney Talks About Santorum, a Bit Mischievously",Mitt Romney notes that Rick Santorum endorsed him in 2008.,Ashley Parker,https://thecaucus.blogs.nytimes.com/2012/01/01/romney-talks-about-santorum-a-bit-mischievously/?searchResultPosition=3
3,"January 1, 2012",Politics,Santorum Pushes Supporters to Keep His Surge Alive,A question of whether his sudden gains in the polls will hold.,Jeremy W. Peters,https://thecaucus.blogs.nytimes.com/2012/01/01/santorum-on-the-rise-pushes-his-supporters-for-help/?searchResultPosition=4
4,"January 1, 2012",Politics,Gingrich to Take Aim at Romney in New Hampshire,Newt Gingrich said on Sunday he intended to campaign in New Hampshire and will criticize Mitt Romney over an abortion provision in the Massachusetts health care program.,Katharine Q. Seelye,https://thecaucus.blogs.nytimes.com/2012/01/01/gingrich-plans-to-target-romney-in-new-hampshire/?searchResultPosition=5
...,...,...,...,...,...,...
132498,March 17,Asia Pacific,Landmark Ruling Cracks Door Open for Same-Sex Marriage in Japan,A court found that it was unconstitutional for the country not to recognize the unions. But change would come only if Parliament passes legislation.,Ben Dooley and Hisako Ueno,https://www.nytimes.com/2021/03/17/world/asia/japan-gay-marriage.html?searchResultPosition=16
132499,March 17,Asia Pacific,U.S. Punishes 24 Chinese Officials on Eve of First Talks Under Biden,"Setting a confrontational tone ahead of meetings in Alaska, the United States punished Chinese officials involved in eroding democracy in Hong Kong.","Steven Lee Myers, Austin Ramzy and Lara Jakes",https://www.nytimes.com/2021/03/17/world/asia/us-china-biden.html?searchResultPosition=17
132500,March 16,Asia Pacific,‘I Had to Prove That I Exist’: Transgender Anchor Makes History in Bangladesh,"Tashnuva Anan Shishir’s triumphant moment came after a long fight for equality that included harassment, bullying and suicide attempts.",Geneva Abdul,https://www.nytimes.com/2021/03/17/world/asia/bangladesh-transgender-tv-anchor.html?searchResultPosition=18
132501,March 16,World,China approves a fifth Covid-19 vaccine.,"The vaccine has several drawbacks. One is that it does not produce many T-cells, which help clear infected cells and prevent them from spreading.",Sui-Lee Wee,https://www.nytimes.com/2021/03/16/world/covid-vaccine-china-hamster-ovaries.html?searchResultPosition=19


In [6]:
url_words = df.href.apply(lambda x: x.replace('https://', '').split('/')[0:-2])
r = re.compile("\d+|www.nytimes.com|world|news|politics|us")
filtered_words = url_words.apply(lambda x: [i for i in x if not r.match(i)])
print(filtered_words[filtered_words.str.len()==2].shape)
print(filtered_words[filtered_words.str.len()==1].shape)
filtered_words[filtered_words.str.len()==1].apply(lambda x:x[0]).unique()

(0,)
(19583,)


array(['thecaucus.blogs.nytimes.com', 'fivethirtyeight.blogs.nytimes.com',
       'india.blogs.nytimes.com', 'thechoice.blogs.nytimes.com',
       'atwar.blogs.nytimes.com', 'rendezvous.blogs.nytimes.com',
       'business', 'affordable-care-act', 'fiscal-crisis',
       'sinosphere.blogs.nytimes.com', 'parenting.blogs.nytimes.com',
       'iht-retrospective.blogs.nytimes.com', 'ferguson',
       'un-general-assembly', 'first-draft', 'live', 'arts', 'reuters',
       'aponline'], dtype=object)

In [5]:
words_exclude= [
'thechoice.blogs.nytimes.com',
'atwar.blogs.nytimes.com',
'business', 
'parenting.blogs.nytimes.com',
'iht-retrospective.blogs.nytimes.com',
'live',
'arts']

regex = '|'.join(words_exclude)
r = re.compile(regex)

In [6]:
row_to_exclude  = filtered_words.apply(lambda x: list(filter(r.match, x))).str.len()>0
filtered_words[row_to_exclude]
# filtered_words[~row_to_exclude]

90                [thechoice.blogs.nytimes.com]
120                   [atwar.blogs.nytimes.com]
221               [thechoice.blogs.nytimes.com]
258                   [atwar.blogs.nytimes.com]
270                   [atwar.blogs.nytimes.com]
                          ...                  
86715     [iht-retrospective.blogs.nytimes.com]
87957     [iht-retrospective.blogs.nytimes.com]
105615                               [business]
108511                               [business]
124303                                   [live]
Name: href, Length: 3541, dtype: object

In [7]:
df = df[~row_to_exclude].reset_index(drop=True)
df.shape

(128962, 6)

In [11]:
# df.to_csv('nytimes_filter_links_2012_2021.csv', index=False)
df=pd.read_csv('nytimes_filter_links_2012_2021.csv',header=[0])
df

,story_date,sec,title,abstract,author,href
0,"January 1, 2012",Politics,Virginia Attorney General Backs Off Ballot Proposal,Virginia’s attorney general on Sunday backed off of a proposal to loosen the state’s ballot access rules to allow more Republican presidential candidates to qualify.,Michael D. Shear,https://thecaucus.blogs.nytimes.com/2012/01/01/virginia-attorney-general-backs-off-ballot-proposal/?searchResultPosition=1
1,"January 1, 2012",Politics,Gingrich Says He’s Been ‘Romney-Boated’,Newt Gingrich promised an aggressive and pointed response in New Hampshire and South Carolina to attack advertisements in Iowa by groups supporting Mitt Romney.,Trip Gabriel,https://thecaucus.blogs.nytimes.com/2012/01/01/gingrich-says-hes-been-romney-boated/?searchResultPosition=2
2,"January 1, 2012",Politics,"Romney Talks About Santorum, a Bit Mischievously",Mitt Romney notes that Rick Santorum endorsed him in 2008.,Ashley Parker,https://thecaucus.blogs.nytimes.com/2012/01/01/romney-talks-about-santorum-a-bit-mischievously/?searchResultPosition=3
3,"January 1, 2012",Politics,Santorum Pushes Supporters to Keep His Surge Alive,A question of whether his sudden gains in the polls will hold.,Jeremy W. Peters,https://thecaucus.blogs.nytimes.com/2012/01/01/santorum-on-the-rise-pushes-his-supporters-for-help/?searchResultPosition=4
4,"January 1, 2012",Politics,Gingrich to Take Aim at Romney in New Hampshire,Newt Gingrich said on Sunday he intended to campaign in New Hampshire and will criticize Mitt Romney over an abortion provision in the Massachusetts health care program.,Katharine Q. Seelye,https://thecaucus.blogs.nytimes.com/2012/01/01/gingrich-plans-to-target-romney-in-new-hampshire/?searchResultPosition=5
...,...,...,...,...,...,...
128957,March 17,Asia Pacific,Landmark Ruling Cracks Door Open for Same-Sex Marriage in Japan,A court found that it was unconstitutional for the country not to recognize the unions. But change would come only if Parliament passes legislation.,Ben Dooley and Hisako Ueno,https://www.nytimes.com/2021/03/17/world/asia/japan-gay-marriage.html?searchResultPosition=16
128958,March 17,Asia Pacific,U.S. Punishes 24 Chinese Officials on Eve of First Talks Under Biden,"Setting a confrontational tone ahead of meetings in Alaska, the United States punished Chinese officials involved in eroding democracy in Hong Kong.","Steven Lee Myers, Austin Ramzy and Lara Jakes",https://www.nytimes.com/2021/03/17/world/asia/us-china-biden.html?searchResultPosition=17
128959,March 16,Asia Pacific,‘I Had to Prove That I Exist’: Transgender Anchor Makes History in Bangladesh,"Tashnuva Anan Shishir’s triumphant moment came after a long fight for equality that included harassment, bullying and suicide attempts.",Geneva Abdul,https://www.nytimes.com/2021/03/17/world/asia/bangladesh-transgender-tv-anchor.html?searchResultPosition=18
128960,March 16,World,China approves a fifth Covid-19 vaccine.,"The vaccine has several drawbacks. One is that it does not produce many T-cells, which help clear infected cells and prevent them from spreading.",Sui-Lee Wee,https://www.nytimes.com/2021/03/16/world/covid-vaccine-china-hamster-ovaries.html?searchResultPosition=19


In [12]:
number_of_chunks= 30

for idx, chunk in enumerate(np.array_split(df, number_of_chunks)):
    chunk.to_csv('nytimes_2012_2021/nytimes_html_%d.csv'%(idx+50), index=False)

In [3]:
headers={
    'accept': '*/*',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh,zh-TW;q=0.9,zh-CN;q=0.8,en;q=0.7',
    'content-type': 'text/plain;charset=UTF-8',
#     'referer': 'https://www.nytimes.com/',
    'sec-ch-ua': '"Chromium";v="88", "Google Chrome";v="88", ";Not A Brand";v="99"',
    'sec-ch-ua-mobile': '?0',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'no-cors',
    'sec-fetch-site': 'same-site',
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36'
}

def get_html(url):
    try:
        coverpage = requests.get(url, headers= headers).content
        time.sleep(2)
        soup = BeautifulSoup(coverpage, 'html.parser')
#         html = soup.find_all('section',{'name':'articleBody'})[0]
        html = str(html)
    except:
        html='error'
        print('error', url)
    return html

def get_html(url):
    driver = webdriver.Chrome(options=chrome_options, executable_path=chrome_drive)
    try:
        driver.get(url)
        time.sleep(1)
        html = driver.page_source
        html = str(html)
        driver.quit()     
    except:
        html='error'
        print('error', url)
    return html

In [13]:
df=pd.read_csv('nytimes_2012_2021/nytimes_html_%d.csv'%50, header=[0])
df

,story_date,sec,title,abstract,author,href
0,"January 1, 2012",Politics,Virginia Attorney General Backs Off Ballot Proposal,Virginia’s attorney general on Sunday backed off of a proposal to loosen the state’s ballot access rules to allow more Republican presidential candidates to qualify.,Michael D. Shear,https://thecaucus.blogs.nytimes.com/2012/01/01/virginia-attorney-general-backs-off-ballot-proposal/?searchResultPosition=1
1,"January 1, 2012",Politics,Gingrich Says He’s Been ‘Romney-Boated’,Newt Gingrich promised an aggressive and pointed response in New Hampshire and South Carolina to attack advertisements in Iowa by groups supporting Mitt Romney.,Trip Gabriel,https://thecaucus.blogs.nytimes.com/2012/01/01/gingrich-says-hes-been-romney-boated/?searchResultPosition=2
2,"January 1, 2012",Politics,"Romney Talks About Santorum, a Bit Mischievously",Mitt Romney notes that Rick Santorum endorsed him in 2008.,Ashley Parker,https://thecaucus.blogs.nytimes.com/2012/01/01/romney-talks-about-santorum-a-bit-mischievously/?searchResultPosition=3
3,"January 1, 2012",Politics,Santorum Pushes Supporters to Keep His Surge Alive,A question of whether his sudden gains in the polls will hold.,Jeremy W. Peters,https://thecaucus.blogs.nytimes.com/2012/01/01/santorum-on-the-rise-pushes-his-supporters-for-help/?searchResultPosition=4
4,"January 1, 2012",Politics,Gingrich to Take Aim at Romney in New Hampshire,Newt Gingrich said on Sunday he intended to campaign in New Hampshire and will criticize Mitt Romney over an abortion provision in the Massachusetts health care program.,Katharine Q. Seelye,https://thecaucus.blogs.nytimes.com/2012/01/01/gingrich-plans-to-target-romney-in-new-hampshire/?searchResultPosition=5
...,...,...,...,...,...,...
4294,"March 19, 2012",World,Rendezvous Always Knew ‘Bilinguals Are Smarter’,"Of course, we already knew this. But it was nice to have an article in The New York Times Sunday Review confirm it. But that’s not why I’m raising my kids trilingual.",Marcus Mabry,https://rendezvous.blogs.nytimes.com/2012/03/19/rendezvous-always-knew-bilinguals-are-smarter/?searchResultPosition=14
4295,"March 19, 2012",Europe,4 Killed at Jewish School in Southwestern France,"A man opened fire outside a Jewish school in southwest France on Monday, and the police said the gun used was linked to two recent attacks.",Scott Sayare and Steven Erlanger,https://www.nytimes.com/2012/03/20/world/europe/gunman-kills-3-at-a-jewish-school-in-france.html?searchResultPosition=15
4296,"March 19, 2012",World,Keeping Alcohol Out of War Zones,"Nearly any combat outpost in Afghanistan is automatically part of a tense and volatile mix, including a hardened enemy, increasingly deadly land mines, powerful weapons, the stress of multiple deployments, searing heat and unfriendly locals.",Mark McDonald,https://rendezvous.blogs.nytimes.com/2012/03/19/when-alcohol-invades-a-war-zone/?searchResultPosition=16
4297,"March 19, 2012",Asia Pacific,Eternal Rhythms Embody a National Spirit,"The Nrityagram Dance Ensemble is a lustrous exemplar of Odissi, one of India’s classical dance forms.",The New York Times,https://india.blogs.nytimes.com/2012/03/19/eternal-rhythms-embody-a-national-spirit/?searchResultPosition=17


In [9]:
pandarallel.initialize(nb_workers=16, progress_bar= False)
batch_size =16
# idx = 50
# print('\n------------  Index: %d --------------\n'%idx)
# df=pd.read_csv('nytimes_2012_2021/nytimes_html_%d.csv'%idx, header=[0])

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [11]:
i=3744
while i<= df.shape[0]:
    print(i, end=',')
    df.loc[i: i+ batch_size-1, 'html'] =\
    df.loc[i: i+ batch_size-1, 'href'].parallel_apply(get_html).values
    i += batch_size

    if i%512==0:
        df.to_csv('nytimes_2012_2021/nytimes_html_%d.csv'%idx, index= False)

df.to_csv('nytimes_2012_2021/nytimes_html_%d.csv'%idx, index= False)

3744,3760,3776,3792,3808,3824,3840,3856,3872,3888,3904,3920,3936,3952,3968,3984,4000,4016,4032,4048,4064,4080,4096,4112,4128,4144,4160,4176,4192,4208,4224,4240,4256,4272,4288,

In [12]:
pandarallel.initialize(nb_workers=16, progress_bar= False)
batch_size =16
for idx in range(51,80):
    print('\n------------  Index: %d --------------\n'%idx)
    df=pd.read_csv('nytimes_2012_2021/nytimes_html_%d.csv'%idx, header=[0])
    df['html']=None
    i=0
    while i<= df.shape[0]:
        print(i, end=',')
        df.loc[i: i+ batch_size-1, 'html'] =\
        df.loc[i: i+ batch_size-1, 'href'].parallel_apply(get_html).values
        i += batch_size

        if i%512==0:
            df.to_csv('nytimes_2012_2021/nytimes_html_%d.csv'%idx, index= False)
    
    df.to_csv('nytimes_2012_2021/nytimes_html_%d.csv'%idx, index= False)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

------------  Index: 51 --------------

0,16,32,48,64,80,96,112,128,144,160,176,192,208,224,240,256,272,288,304,320,336,352,368,384,400,416,432,448,464,480,496,512,528,544,560,576,592,608,624,640,656,672,688,704,720,736,752,768,784,800,816,832,848,864,880,896,912,928,944,960,976,992,1008,1024,1040,1056,1072,1088,1104,1120,1136,1152,1168,1184,1200,1216,1232,1248,1264,1280,1296,1312,1328,1344,1360,1376,1392,1408,1424,1440,1456,1472,1488,1504,1520,1536,1552,1568,1584,1600,1616,1632,1648,1664,1680,1696,1712,1728,1744,1760,1776,1792,1808,error https://india.blogs.nytimes.com/2012/04/25/s-p-gives-negative-outlook-for-india/?searchResultPosition=15
1824,error https://www.nytimes.com/2012/04/25/world/asia/us-warns-north-korea-on-nuclear-test.html?searchResultPosition=27
1840,1856,1872,1888,1904,1920,1936,1952,1968,1984,2000,2016,2032,2048,2064,error ht

WebDriverException: Message: Service ./chromedriver unexpectedly exited. Status code was: 1


In [17]:
pandarallel.initialize(nb_workers=8, progress_bar= False)
batch_size =8
while i<= df.shape[0]:
    print(i, end=',')
    df.loc[i: i+ batch_size-1, 'html'] =\
    df.loc[i: i+ batch_size-1, 'href'].parallel_apply(get_html).values
    i += batch_size

    if i%512==0:
        df.to_csv('nytimes_2012_2021/nytimes_html_%d.csv'%idx, index= False)

df.to_csv('nytimes_2012_2021/nytimes_html_%d.csv'%idx, index= False)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
672,errorerrorerror   https://www.nytimes.com/2013/07/30/us/court-weighs-exclusion-of-jurors-because-theyre-gay.html?searchResultPosition=4errorhttps://www.nytimes.com/2013/07/29/us/garry-davis-man-of-no-nation-dies-at-91.html?searchResultPosition=9https://www.nytimes.com/2013/07/30/us/politics/obama-meetings.html?searchResultPosition=5error


  https://www.nytimes.com/2013/07/30/us/politics/at-the-11th-hour-a-languid-congress.html?searchResultPosition=7https://www.nytimes.com/2013/07/30/us/politics/william-scranton-former-pennsylvania-governor-dies-at-96.html?searchResultPosition=6

errorerror  https://www.nytimes.com/2013/07/30/us/too-much-rain-in-the-south-too-little-produce-on-the-shelves.html?searchResultPosition=3https://www.nytimes.com/2013/07/30/us/us-pushes-for-global-eye-on-south-sudan-conflict.html?searchResultPosition=8

680,688,696,7

In [18]:
pandarallel.initialize(nb_workers=8, progress_bar= False)
batch_size =8
for idx in range(57,80):
    print('\n------------  Index: %d --------------\n'%idx)
    df=pd.read_csv('nytimes_2012_2021/nytimes_html_%d.csv'%idx, header=[0])
    df['html']=None
    i=0
    while i<= df.shape[0]:
        print(i, end=',')
        df.loc[i: i+ batch_size-1, 'html'] =\
        df.loc[i: i+ batch_size-1, 'href'].parallel_apply(get_html).values
        i += batch_size

        if i%512==0:
            df.to_csv('nytimes_2012_2021/nytimes_html_%d.csv'%idx, index= False)
    
    df.to_csv('nytimes_2012_2021/nytimes_html_%d.csv'%idx, index= False)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

------------  Index: 57 --------------

0,8,16,24,32,40,48,56,64,72,80,88,96,104,112,120,128,136,144,152,160,168,176,184,192,200,208,216,224,232,240,248,256,264,272,280,288,296,304,312,320,328,336,344,352,360,368,376,384,392,400,408,416,424,432,440,448,456,464,472,480,488,496,504,512,520,528,536,544,552,560,568,576,584,592,600,608,616,624,632,640,648,656,664,672,680,688,696,704,712,720,728,736,744,752,760,768,776,784,792,800,808,816,824,832,840,848,856,864,872,880,888,896,904,912,920,928,936,944,952,960,968,976,984,992,1000,1008,1016,1024,1032,1040,1048,1056,1064,1072,1080,1088,1096,1104,1112,1120,1128,1136,1144,1152,1160,1168,1176,1184,1192,1200,1208,1216,1224,1232,1240,1248,1256,1264,1272,1280,1288,1296,1304,1312,1320,1328,1336,1344,1352,1360,1368,1376,1384,1392,1400,1408,1416,1424,1432,1440,1448,1456,1464,1472,1480,1488,1496,1504,1512,1520,15

KeyboardInterrupt: 

https://www.nytimes.com/2013/12/28/world/asia/india-consular-employee-retaliation.html?searchResultPosition=3https://india.blogs.nytimes.com/2013/12/27/a-conversation-with-aam-aadmi-party-leader-arvind-kejriwal/?searchResultPosition=7



Process ForkPoolWorker-31413:
Process ForkPoolWorker-31410:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/yibo/miniconda3/envs/py36/lib/python3.6/multiprocessing/pool.py", line 108, in worke

In [19]:
df.to_csv('nytimes_2012_2021/nytimes_html_%d.csv'%idx, index= False)

In [2]:
from newspaper import Article

def get_html(url):
    try:
        article = Article(url)
        time.sleep(2)
        article.download()
        html = article.html 
    except:
        html='error'
        print('error', url)
    return html

In [7]:
idx =57
df=pd.read_csv('nytimes_2012_2021/nytimes_html_%d.csv'%idx, header=[0])
df

,story_date,sec,title,abstract,author,href,html
0,"November 8, 2013",Asia Pacific,"Reform, Ahoy? A Newspaper Heralds New Horizons for the Chinese Ship of State","A day before a top-level political meeting opens in China, a prominent newspaper highlights widespread calls for extensive reform, including social and political change, in a hefty special supplement.",Didi Kirsten Tatlow,https://sinosphere.blogs.nytimes.com/2013/11/08/reform-ahoy-a-newspaper-heralds-new-horizons-for-the-chinese-ship-of-state/?searchResultPosition=14,"<html lang=""en"" class=""flag-limitFabrikSave-on flag-facebookOauthButton-on flag-videoVHSCover-on flag-videoVHSShareTools-on flag-videoVHSEmbeddedOnly-on flag-removeInternationalEdition-on flag-piiBlockDFP-on flag-indexAsHeaderBidder-on flag-caslOpt-on desktop has-comments blog-sinosphere section-world wf-active viewport-small viewport-small-10 viewport-small-20 viewport-medium viewport-medium..."
1,"November 8, 2013",Asia Pacific,"As Plenum Starts, an Economist Sees Crisis Ahead","China faces a crisis unless big economic changes are accompanied by a political overhaul, a respected Chinese economist says as Communist Party leaders prepare to meet on the nation’s economic future.",Chris Buckley,https://sinosphere.blogs.nytimes.com/2013/11/08/as-plenum-starts-an-economist-sees-crisis-ahead/?searchResultPosition=15,<html><head>\n <title>503 first byte timeout</title>\n </head>\n <body>\n <h1>Error 503 first byte timeout</h1>\n <p>first byte timeout</p>\n <h3>Guru Mediation:</h3>\n <p>Details: cache-dal21245-DAL 1616199401 2538404783</p>\n <hr>\n <p>Varnish cache server</p>\n \n\n</body></html>
2,"November 8, 2013",Asia Pacific,Facing a Flood of Junk Text Messages in China,Measures to curb unwanted text messages would have widespread public support in a country with 1.2 billion mobile subscribers and where each mobile number receives an average of one junk text a day.,Austin Ramzy,https://sinosphere.blogs.nytimes.com/2013/11/08/facing-a-flood-of-junk-text-messages-in-china/?searchResultPosition=16,"<html lang=""en"" class=""flag-limitFabrikSave-on flag-facebookOauthButton-on flag-videoVHSCover-on flag-videoVHSShareTools-on flag-videoVHSEmbeddedOnly-on flag-removeInternationalEdition-on flag-piiBlockDFP-on flag-indexAsHeaderBidder-on flag-caslOpt-on desktop has-comments blog-sinosphere section-world wf-active viewport-small viewport-small-10 viewport-small-20 viewport-medium viewport-medium..."
3,"November 8, 2013",Asia Pacific,A Powerful Typhoon Speeds Across the Philippines,"The storm, named Haiyan, killed at least 100 people and forced the evacuation of hundreds of thousands.",Floyd Whaley,https://www.nytimes.com/2013/11/09/world/asia/powerful-typhoon-causes-mass-disruption-in-philippines.html?searchResultPosition=17,"<html lang=""en"" class=""story nytapp-vi-article"" xmlns:og=""http://opengraphprotocol.org/schema/"" data-rh=""lang,class""><head>\n <title>A Powerful Typhoon Speeds Across the Philippines - The New York Times</title>\n <meta data-rh=""true"" property=""article:published_time"" content=""2013-11-08T05:07:46.000Z""><meta data-rh=""true"" property=""article:modified_time"" content=""2013-11-09T03:35:30.000Z..."
4,"November 7, 2013",Asia Pacific,Police Arrest Suspect in Northern China Blasts,"The bombing Wednesday raised concerns about security in China, particularly in the wake of the deadly attack at the entrance to the Forbidden City the week before.",Gerry Mullany,https://sinosphere.blogs.nytimes.com/2013/11/07/police-arrest-suspect-in-northern-china-blasts/?searchResultPosition=18,"<html lang=""en"" class=""flag-limitFabrikSave-on flag-facebookOauthButton-on flag-videoVHSCover-on flag-videoVHSShareTools-on flag-videoVHSEmbeddedOnly-on flag-removeInternationalEdition-on flag-piiBlockDFP-on flag-indexAsHeaderBidder-on flag-caslOpt-on desktop has-comments blog-sinosphere section-world wf-active viewport-small viewport-small-10 viewport-small-20 viewport-medium viewport-medium..."
...,...,..

In [11]:
pandarallel.initialize(nb_workers= 16, progress_bar= False)
batch_size = 16*10

i = 1600

while i<= df.shape[0]:
    print(i, end=',')
    df.loc[i: i+ batch_size-1, 'html'] =\
    df.loc[i: i+ batch_size-1, 'href'].parallel_apply(get_html).values
    i += batch_size

    if i% (batch_size * 20) ==0:
        df.to_csv('nytimes_2012_2021/nytimes_html_%d.csv'%idx, index= False)

df.to_csv('nytimes_2012_2021/nytimes_html_%d.csv'%idx, index= False)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
1600,1760,1920,2080,2240,2400,2560,2720,2880,3040,3200,3360,3520,3680,3840,4000,4160,

In [ ]:
pandarallel.initialize(nb_workers= 16, progress_bar= False)
batch_size = 16*10

for idx in range(58,70):
    print('\n------------  Index: %d --------------\n'%idx)
    df=pd.read_csv('nytimes_2012_2021/nytimes_html_%d.csv'%idx, header=[0])
    df['html']=None
    i=0
    while i<= df.shape[0]:
        print(i, end=',')
        df.loc[i: i+ batch_size-1, 'html'] =\
        df.loc[i: i+ batch_size-1, 'href'].parallel_apply(get_html).values
        i += batch_size

        if i%(batch_size * 20) ==0:
            df.to_csv('nytimes_2012_2021/nytimes_html_%d.csv'%idx, index= False)
    
    df.to_csv('nytimes_2012_2021/nytimes_html_%d.csv'%idx, index= False)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

------------  Index: 58 --------------

0,160,320,480,640,800,960,1120,1280,1440,1600,1760,1920,2080,2240,2400,2560,2720,2880,3040,3200,3360,3520,3680,3840,4000,4160,
------------  Index: 59 --------------

0,160,320,480,640,800,960,1120,1280,1440,1600,1760,1920,2080,2240,2400,2560,2720,2880,3040,3200,3360,3520,3680,3840,4000,4160,
------------  Index: 60 --------------

0,160,320,480,640,800,960,1120,1280,1440,1600,1760,1920,2080,2240,2400,2560,2720,2880,3040,

In [3]:
pandarallel.initialize(nb_workers= 16, progress_bar= False)
batch_size = 16*10

for idx in range(60,70):
    print('\n------------  Index: %d --------------\n'%idx)
    df=pd.read_csv('nytimes_2012_2021/nytimes_html_%d.csv'%idx, header=[0])
    df['html']=None
    i=0
    while i<= df.shape[0]:
        print(i, end=',')
        df.loc[i: i+ batch_size-1, 'html'] =\
        df.loc[i: i+ batch_size-1, 'href'].parallel_apply(get_html).values
        i += batch_size

        if i%(batch_size * 20) ==0:
            df.to_csv('nytimes_2012_2021/nytimes_html_%d.csv'%idx, index= False)
    
    df.to_csv('nytimes_2012_2021/nytimes_html_%d.csv'%idx, index= False)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.

------------  Index: 60 --------------

0,160,320,480,640,800,960,1120,1280,1440,1600,1760,1920,2080,2240,2400,2560,2720,2880,3040,3200,3360,3520,3680,3840,4000,4160,
------------  Index: 61 --------------

0,160,320,480,640,800,960,1120,1280,1440,1600,1760,1920,2080,2240,2400,2560,2720,2880,3040,3200,3360,3520,3680,3840,4000,4160,
------------  Index: 62 --------------

0,160,320,480,640,800,960,1120,1280,1440,1600,1760,1920,2080,2240,2400,2560,2720,2880,3040,3200,3360,3520,3680,3840,4000,4160,
------------  Index: 63 --------------

0,160,320,480,640,800,960,1120,1280,1440,1600,1760,1920,2080,2240,2400,2560,2720,2880,3040,3200,3360,3520,3680,3840,4000,4160,
------------  Index: 64 --------------

0,160,320,480,640,800,960,1120,1280,1440,1600,1760,1920,2080,2240,2400,2560,2720,2880,3040,3200,3360,3520,3680,3840,4000,4160,
------------  Index: 

In [4]:
quit()

In [3]:
url = [
'https://www.nytimes.com/topic/subject/crime-and-criminals',
'https://www.nytimes.com/topic/subject/terrorism',
'https://www.nytimes.com/topic/subject/human-rights-and-human-rights-violations',
'https://www.nytimes.com/topic/subject/defense-and-military-forces',
'https://www.nytimes.com/topic/subject/war-crimes-genocide-and-crimes-against-humanity']

df = pd.DataFrame(url, columns = ['url'])
df[['topic']] = df['url'].str.split('/').apply(lambda x: x[-1])
df

,url,topic
0,https://www.nytimes.com/topic/subject/crime-an...,crime-and-criminals
1,https://www.nytimes.com/topic/subject/terrorism,terrorism
2,https://www.nytimes.com/topic/subject/human-ri...,human-rights-and-human-rights-violations
3,https://www.nytimes.com/topic/subject/defense-...,defense-and-military-forces
4,https://www.nytimes.com/topic/subject/war-crim...,war-crimes-genocide-and-crimes-against-humanity


In [ ]:
df

In [34]:
def get_html_pages(url):
    driver = webdriver.Chrome(options=chrome_options, executable_path=chrome_drive)
    driver.get(url)
    i=0
    while i<10:
        driver.execute_script("window.scrollBy(0,300)")
        time.sleep(1)
        i +=1
    return driver.page_source

In [5]:
pandarallel.initialize()
df[['html']]= df['url'].parallel_apply(get_html_pages).values

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [6]:
df

,url,topic,html
0,https://www.nytimes.com/topic/subject/crime-an...,crime-and-criminals,"<html lang=""en"" xmlns:og=""http://opengraphprot..."
1,https://www.nytimes.com/topic/subject/terrorism,terrorism,"<html lang=""en"" xmlns:og=""http://opengraphprot..."
2,https://www.nytimes.com/topic/subject/human-ri...,human-rights-and-human-rights-violations,"<html lang=""en"" xmlns:og=""http://opengraphprot..."
3,https://www.nytimes.com/topic/subject/defense-...,defense-and-military-forces,"<html lang=""en"" xmlns:og=""http://opengraphprot..."
4,https://www.nytimes.com/topic/subject/war-crim...,war-crimes-genocide-and-crimes-against-humanity,"<html lang=""en"" xmlns:og=""http://opengraphprot..."


In [32]:
for i in range(5):
    html = df.html[i]
    soup = BeautifulSoup(html, 'html.parser')
    find_li = soup.find_all('li', class_= 'css-ye6x8s')
    print(len(find_li))

110
109
110
110
110


In [14]:
find_li[0].find_all('a', href=True)[0]['href']

'/2021/03/09/us/illinois-prison-guard-civil-rights.html'

In [28]:
find_li[0].find_all('span', 'data-testid'=='todays-date')[0].text

'Neil Vigdor'

In [31]:
find_li[0].find_all('div', class_= 'css-1lc2l26 e1xfvim33')[0].text

'March 9, 2021'

In [15]:
new_url = 'https://www.nytimes.com'+ '/2021/03/09/us/illinois-prison-guard-civil-rights.html'
coverpage = requests.get(new_url).content
story = BeautifulSoup(coverpage, 'html.parser')


'\n\n\nIllinois Prison Guard Pleads Guilty for Role in Beating Inmate to Death - The New York Times\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSectionsSEARCHSkip to contentSkip to site indexU.S.Log inToday’s PaperU.S.|Illinois Prison Guard Pleads Guilty for Role in Beating Inmate to DeathAdvertisementContinue reading the main storySupported byContinue reading the main storyIllinois Prison Guard Pleads Guilty for Role in Beating Inmate to DeathWillie Hedden is one of three guards at the Western Illinois Correctional Center charged in the May 2018 assault of Larry Earvin, who was handcuffed at the time.Larry Earvin, in an undated photo, was an inmate at the Western Illinois Correctional Center. A forensic pathologist ruled his death a homicide as a result of blunt-force trauma.Credit...Illinois Department of Corrections, via Associated PressBy Neil VigdorMarch 9, 2021A state prison guard in Illinois pleaded guilty on Tuesday to two counts of federal civil rights violations and one count of

In [25]:
headline = story.find_all('h1', 'data-test-id' == "headline")[0].text
print(headline)
print()

summary = story.find_all('p', id = 'article-summary')[0].text
print(summary)
print()

text = story.find_all('p', class_= 'css-axufdj evys1bk0')
for i in text:
    print(i.text)

Illinois Prison Guard Pleads Guilty for Role in Beating Inmate to Death

Willie Hedden is one of three guards at the Western Illinois Correctional Center charged in the May 2018 assault of Larry Earvin, who was handcuffed at the time.

A state prison guard in Illinois pleaded guilty on Tuesday to two counts of federal civil rights violations and one count of conspiracy to engage in misleading conduct in the 2018 beating death of an inmate who had been handcuffed, prosecutors said.
The guard, Sgt. Willie Hedden, could face up to life in prison on the civil rights counts, but avoided three obstruction charges as part of a plea deal that was reached during a videoconference hearing held by the U.S. District Court in Springfield, Ill.
Under the terms of that agreement, The Associated Press reported, Sergeant Hedden is expected to cooperate with the prosecution’s case against two other guards who were charged in the May 17, 2018, assault of Larry Earvin at the Western Illinois Correctional 